General import statements. REMEMBER, DEEPLIFT_DIR needs to point to the deeplift directory WITHIN the deeplift repo

In [1]:
%matplotlib inline
from __future__ import division;
from __future__ import print_function;
from __future__ import absolute_import;
import sys, os;
from collections import OrderedDict, namedtuple;
import numpy as np;

#Make sure the directory is set to import the lab's version of keras
scriptsDir = os.environ.get("KERAS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable KERAS_DIR");
sys.path.insert(0,scriptsDir)

scriptsDir = os.environ.get("ENHANCER_SCRIPTS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable ENHANCER_SCRIPTS_DIR to point to the enhancer_prediction_code repo");
sys.path.insert(0,scriptsDir+"/featureSelector/deepLIFFT");
from deepLIFTutils import makePngOfSequenceDeepLIFTScores

import deeplift
import deeplift.conversion.keras_conversion as kc

Using Theano backend.
Using gpu device 0: GeForce GT 750M (CNMeM is disabled, cuDNN not available)
/Users/avantishrikumar/anaconda/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


Load the keras model

In [2]:
#Load the keras model, make sure you normalise the weights
#of the first convolutional layer to be mean-centered at each position.
model_weights = "modelsDir_100Kruns/record_0_model_jFhL6_modelWeights.h5"
model_yaml = "modelsDir_100Kruns/record_0_model_jFhL6_modelYaml.yaml"
reload(kc)
keras_model = kc.load_keras_model(model_weights, model_yaml, normalise_conv_for_one_hot_encoded_input=False)

Load the data

In [3]:
scriptsDir = os.environ.get("UTIL_SCRIPTS_DIR");
if (scriptsDir is None):
    raise Exception("Please set environment variable UTIL_SCRIPTS_DIR to point to the deeplift code");
sys.path.insert(0,scriptsDir);
from importDataPackage import importData
reload(importData)
trainData, validData, testData = importData.loadTrainTestValidFromYaml("yaml_100K/features.yaml",
                                                                       "yaml_100K/labels.yaml",
                                                                       "yaml_100K/splits.yaml")

(0, 'rows skipped from', 'features_100K.gz')
Returning desired dict
Making numpy arrays out of the loaded files
('train', 'shapeX', (80000, 50))
('train', 'shapeY', (80000, 45))
('valid', 'shapeX', (10000, 50))
('valid', 'shapeY', (10000, 45))
('test', 'shapeX', (10000, 50))
('test', 'shapeY', (10000, 45))


In [4]:
data = trainData.concat(validData, testData)

Convert the keras sequential model into a deeplift sequential model, and compile the functions to compute the contributions and multipliers - the multipliers are analogous to the gradients

In [5]:
from deeplift.blobs import MxtsMode
reload(kc)

<module 'deeplift.conversion.keras_conversion' from '/Users/avantishrikumar/Research/deeplift/deeplift/conversion/keras_conversion.pyc'>

In [6]:
deeplift_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.DeepLIFT)
deeplift_contribs_func = deeplift_model.get_target_contribs_func(find_scores_layer_idx=0)
deeplift_multipliers_func = deeplift_model.get_target_multipliers_func(find_scores_layer_idx=0)

Do the same for other saliency map functions

In [24]:
gradients_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.Gradient)
grad_times_inp_func = gradients_model.get_target_contribs_func(find_scores_layer_idx=0)
grad_func = gradients_model.get_target_multipliers_func(find_scores_layer_idx=0)
guided_backprop_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.GuidedBackprop)
guided_backprop_times_inp_func = guided_backprop_model.get_target_contribs_func(find_scores_layer_idx=0)
guided_backprop_func = guided_backprop_model.get_target_multipliers_func(find_scores_layer_idx=0)
guided_backprop_deeplift_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.GuidedBackpropDeepLIFT)
guided_backprop_deeplift_times_inp_func = guided_backprop_deeplift_model.get_target_contribs_func(find_scores_layer_idx=0)
guided_backprop_deeplift_func = guided_backprop_deeplift_model.get_target_multipliers_func(find_scores_layer_idx=0)
deconv_model = kc.convert_sequential_model(keras_model, mxts_mode=MxtsMode.DeconvNet)
deconv_func = deconv_model.get_target_multipliers_func(find_scores_layer_idx=0)
deconv_times_inp_func = deconv_model.get_target_contribs_func(find_scores_layer_idx=0)

In [25]:
import theano
predictions_func = theano.function([deeplift_model.get_layers()[0].get_activation_vars()],
                                   deeplift_model.get_layers()[-1].get_activation_vars(),
                                   allow_input_downcast=True)

In [26]:
predictions_func([np.ones(testData.X[0].shape)*0.001])

array([[ 0.00719933,  0.00894803,  0.01022888,  0.00756006,  0.00769961,
         0.00704683,  0.00673796,  0.00914059,  0.00965482,  0.00790531,
         0.00867973,  0.00711914,  0.00629144,  0.00613344,  0.00732677,
         0.00700838,  0.0096201 ,  0.01024602,  0.00887473,  0.00915498,
         0.00766505,  0.00867055,  0.00903935,  0.00873579,  0.00890495,
         0.00911928,  0.0095769 ,  0.00908662,  0.00882156,  0.01010747,
         0.00655401,  0.00731878,  0.00805646,  0.00802046,  0.00947253,
         0.00848896,  0.00730444,  0.00761912,  0.01101029,  0.00731911,
         0.0079971 ,  0.00846429,  0.0073612 ,  0.00946271,  0.00914535]], dtype=float32)

In [27]:
predictions = predictions_func(data.X)

In [28]:
for output_idx in range(data.Y.shape[-1]):
    print(sorted(enumerate(zip(deeplift_model.get_layers()[-2].W[:,output_idx],
                               deeplift_model.get_layers()[-4].b)), key=lambda x: -abs(x[1][0]) )[0:2])


[(14, (-19.807505, 0.30824822)), (5, (-19.453634, 0.30825487))]
[(5, (-19.765078, 0.30825487)), (10, (-15.632283, 0.19002751))]
[(5, (-19.546129, 0.30825487)), (12, (-17.958994, 0.24949609))]
[(7, (-19.473919, 0.30306029)), (5, (-19.339251, 0.30825487))]
[(5, (-19.665556, 0.30825487)), (17, (-16.285915, 0.19980519))]
[(5, (-19.814039, 0.30825487)), (11, (-17.587646, 0.18616237))]
[(19, (-20.242025, 0.30106205)), (5, (-19.573761, 0.30825487))]
[(8, (-19.359825, 0.30875102)), (5, (-19.180332, 0.30825487))]
[(9, (-19.739803, 0.29593748)), (5, (-19.521702, 0.30825487))]
[(14, (-20.017479, 0.30824822)), (0, (-15.859643, 0.20394446))]
[(14, (-19.884604, 0.30824822)), (12, (-18.430599, 0.24949609))]
[(14, (-19.805882, 0.30824822)), (7, (-19.589598, 0.30306029))]
[(14, (-20.625605, 0.30824822)), (17, (-16.758018, 0.19980519))]
[(14, (-20.32843, 0.30824822)), (11, (-16.461458, 0.18616237))]
[(19, (-19.894787, 0.30106205)), (14, (-19.87229, 0.30824822))]
[(14, (-19.687641, 0.30824822)), (8, (-19

In [29]:
for i in range(deeplift_model.get_layers()[1].W.shape[1]):
    print("node:",i)
    print(sorted(enumerate(deeplift_model.get_layers()[1].W[:,i]), key=lambda x: -np.abs(x[1]))[0:5])

node: 0
[(10, -4.5617361), (11, -4.5606618), (12, -4.5579462), (13, -4.5523386), (14, -4.5269675)]
node: 1
[(21, 0.38230917), (20, 0.35195157), (49, 0.34952912), (22, 0.31976384), (23, 0.31735089)]
node: 2
[(7, 0.27965769), (6, 0.25055015), (9, 0.24158125), (44, 0.19970311), (29, 0.18103105)]
node: 3
[(25, -3.3250067), (28, -3.319689), (29, -3.3139145), (26, -3.3063612), (27, -3.3011355)]
node: 4
[(7, 0.25213218), (5, 0.24673554), (18, 0.18748948), (6, 0.18005429), (9, 0.16755639)]
node: 5
[(1, -5.764812), (3, -5.7621217), (0, -5.7486353), (4, -5.7479744), (2, -5.7422199)]
node: 6
[(27, -2.6288085), (26, -2.6136553), (29, -2.5932529), (28, -2.577121), (25, -2.5742512)]
node: 7
[(24, -5.7993035), (23, -5.795094), (22, -5.7878399), (21, -5.7834282), (20, -5.7756348)]
node: 8
[(40, -5.7506771), (44, -5.7505345), (41, -5.7465501), (43, -5.7456746), (42, -5.7349639)]
node: 9
[(49, -5.7643399), (46, -5.7599635), (48, -5.7537646), (45, -5.7519193), (47, -5.7518249)]
node: 10
[(14, -4.4802117)

Compute the contributions for all 3 tasks and the multipliers for the third task

In [32]:
deeplift_contribs,\
deeplift_multipliers,\
grad_times_inp,\
grad,\
guided_backprop_times_inp,\
guided_backprop,\
guided_backprop_deeplift_times_inp,\
guided_backprop_deeplift,\
deconv_times_inp,\
deconv = [[np.array(contribs_func(task_idx=i, input_data_list=[data.X], batch_size=1000, progress_update=None))
                    for i in range(data.Y.shape[-1])]
                    for contribs_func in [deeplift_contribs_func,
                                          deeplift_multipliers_func,
                                          grad_times_inp_func,
                                          grad_func,
                                          guided_backprop_times_inp_func,
                                          guided_backprop_func,
                                          guided_backprop_deeplift_times_inp_func,
                                          guided_backprop_deeplift_func,
                                          deconv_times_inp_func,
                                          deconv_func]]

In [33]:
scores = np.array(deeplift_contribs)
for scores_name, scores in [('deeplift',deeplift_contribs),
                            ('grad_times_inp', grad_times_inp),
                            ('grad', grad),
                            ('guided_backprop_times_inp', guided_backprop_times_inp),
                            ('guided_backprop', guided_backprop),
                            ('guided_backprop_deeplift_times_inp', guided_backprop_deeplift_times_inp),
                            ('guided_backprop_deeplift', guided_backprop_deeplift),
                            ('deconv_times_inp', deconv_times_inp),
                            ('deconv', deconv)]:
    num_caps = 10
    inputs_per_cap = int(data.X.shape[-1]/num_caps)
    task_idx = 0
    true_positives = data.Y*(np.array(predictions)>0.5)
    average_proportion_on_correct_array = []
    for cap1 in range(num_caps):
        for cap2 in range(cap1+1,num_caps):
            relevant_inputs_cap1 = slice(cap1*inputs_per_cap,(cap1+1)*inputs_per_cap)
            relevant_inputs_cap2 = slice(cap2*inputs_per_cap,(cap2+1)*inputs_per_cap)
            true_positives_mask = true_positives[:,task_idx]
            
            scores_on_true_positives = np.compress(np.nonzero(true_positives_mask)[0],scores[task_idx],axis=0)
            input_data_true_positives = np.compress(np.nonzero(true_positives_mask)[0], data.X, axis=0)
            
            scores_relevant_inputs_cap1 = scores_on_true_positives[:,relevant_inputs_cap1]
            scores_relevant_inputs_cap2 = scores_on_true_positives[:,relevant_inputs_cap2]
            signs_relevant_inputs_cap1 = np.sign(np.compress(np.nonzero(true_positives_mask)[0], data.X[:,relevant_inputs_cap1], axis=0))
            signs_relevant_inputs_cap2 = np.sign(np.compress(np.nonzero(true_positives_mask)[0], data.X[:,relevant_inputs_cap2], axis=0))
            correct_sign_scores_cap1 = np.abs(scores_relevant_inputs_cap1*(np.equal(np.sign(scores_relevant_inputs_cap1),
                                                                                    signs_relevant_inputs_cap1)))
            correct_sign_scores_cap2 = np.abs(scores_relevant_inputs_cap2*(np.equal(np.sign(scores_relevant_inputs_cap2),
                                                                                    signs_relevant_inputs_cap2)))
            
            
            total_scores = np.sum(np.abs(scores_on_true_positives), axis=1)
            scores_on_relevant_indices = np.sum(correct_sign_scores_cap1, axis=1) +\
                                         np.sum(correct_sign_scores_cap2, axis=1)
            total_scores = 0.0000001*(total_scores == 0) + total_scores #prevent div by zero
            proportion_on_correct_indices = scores_on_relevant_indices/total_scores
            average_proportion = np.mean(proportion_on_correct_indices)
            average_proportion_on_correct_array.append(average_proportion)
            task_idx += 1
    print(scores_name, average_proportion)

deeplift 0.9781520116
grad_times_inp 0.8175276112
grad 0.298641559752
guided_backprop_times_inp 0.143816033779
guided_backprop 0.0698287201052
guided_backprop_deeplift_times_inp 0.144306206722
guided_backprop_deeplift 0.0699108552511
deconv_times_inp 0.091537127847
deconv 0.0443167054198
